# Delta Lake Change Column Name

In [2]:
import pyspark
from delta import *

In [3]:
builder = (
    pyspark.sql.SparkSession.builder.appName("MyApp")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config(
        "spark.sql.catalog.spark_catalog",
        "org.apache.spark.sql.delta.catalog.DeltaCatalog",
    )
)

In [4]:
spark = configure_spark_with_delta_pip(builder).getOrCreate()

:: loading settings :: url = jar:file:/Users/matthew.powers/opt/miniconda3/envs/pyspark-330-delta-210/lib/python3.9/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/matthew.powers/.ivy2/cache
The jars for the packages stored in: /Users/matthew.powers/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-11b6b81a-37b8-45cf-b21a-6692e302539a;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.1.0 in central
	found io.delta#delta-storage;2.1.0 in central
	found org.antlr#antlr4-runtime;4.8 in central
	found org.codehaus.jackson#jackson-core-asl;1.9.13 in central
:: resolution report :: resolve 320ms :: artifacts dl 13ms
	:: modules in use:
	io.delta#delta-core_2.12;2.1.0 from central in [default]
	io.delta#delta-storage;2.1.0 from central in [default]
	org.antlr#antlr4-runtime;4.8 from central in [default]
	org.codehaus.jackson#jackson-core-asl;1.9.13 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number|

22/09/12 10:51:21 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [5]:
columns = ["letter", "number"]
data = [("a", 1), ("b", 2), ("c", 3)]
rdd = spark.sparkContext.parallelize(data)
df = rdd.toDF(columns)

In [6]:
spark.sql("DROP TABLE IF EXISTS my_little_table")

DataFrame[]

In [8]:
df.write.saveAsTable("my_little_table")

In [9]:
spark.sql(
    """ALTER TABLE `my_little_table` SET TBLPROPERTIES (
   'delta.columnMapping.mode' = 'name',
   'delta.minReaderVersion' = '2',
   'delta.minWriterVersion' = '5')"""
)

DataFrame[]

In [14]:
spark.sql("ALTER TABLE `my_little_table` RENAME TO `whatever`")

DataFrame[]

In [16]:
spark.sql("SELECT * FROM `whatever`").show()

+------+------+
|letter|number|
+------+------+
|     c|     3|
|     a|     1|
|     b|     2|
+------+------+



In [18]:
spark.sql("ALTER TABLE `whatever` ALTER COLUMN `number` FIRST")

AnalysisException: ALTER COLUMN ... FIRST | ALTER is only supported with v2 tables.